In [16]:
import numpy as np 
import pandas as pd 
import tensorflow
import cv2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten,BatchNormalization,Dense,Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16,preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import img_to_array,load_img


In [7]:
image_size=[32,32,3]
model=VGG16(input_shape=image_size,weights='imagenet',include_top=False)

58900480/58889256 [==============================] - 0s 0us/step


In [8]:
for layers in model.layers : 
    layers.trainable = False

In [9]:
x=Flatten()(model.output)
output=Dense(10,activation='softmax')(x)

In [10]:
model1=Model(inputs=model.input,outputs=output)
model1.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 32, 32, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 32, 32, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 16, 16, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 16, 16, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 16, 16, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 8, 8, 128)         0     

In [20]:
(x_train, y_train), (x_test, y_test) = tensorflow.keras.datasets.cifar10.load_data()

170508288/170498071 [==============================] - 2s 0us/step


In [21]:
x_train.shape,y_train.shape,x_test.shape,y_test.shape

((50000, 32, 32, 3), (50000, 1), (10000, 32, 32, 3), (10000, 1))

In [22]:
pd.DataFrame(y_train).value_counts()

0    5000
1    5000
2    5000
3    5000
4    5000
5    5000
6    5000
7    5000
8    5000
9    5000
dtype: int64

In [23]:
x_train = x_train[0:20000]
y_train = y_train[0:20000]

In [24]:
pd.DataFrame(y_train).value_counts()

2    2042
6    2038
7    2027
8    2012
3    2011
4    2009
0    1989
1    1981
9    1966
5    1925
dtype: int64

In [25]:
model1.compile(optimizer='adam' , loss='categorical_crossentropy' , metrics =['accuracy'])


In [26]:
x_train = np.array([cv2.resize(x , (32,32), interpolation = cv2.INTER_AREA) for x in x_train])
x_test = np.array([cv2.resize(x , (32,32), interpolation = cv2.INTER_AREA) for x in x_test])

In [27]:
train_datagen = ImageDataGenerator(rescale = 1./255)
train_datagen.fit(x_train)
test_datagen = ImageDataGenerator(rescale = 1./255)
test_datagen.fit(x_test)

In [28]:
train_data = train_datagen.flow(x_train,y_train , batch_size = 20)
test_data = test_datagen.flow(x_test , y_test , batch_size = 20)

In [30]:
history = model1.fit(train_data , validation_data=test_data, epochs = 10)

Epoch 1/10
1000/1000 [==============================] - 293s 293ms/step - loss: 115.4704 - accuracy: 0.0233 - val_loss: 114.9937 - val_accuracy: 0.0276
Epoch 2/10
1000/1000 [==============================] - 292s 292ms/step - loss: 114.6958 - accuracy: 0.0286 - val_loss: 114.4008 - val_accuracy: 0.0325
Epoch 3/10
1000/1000 [==============================] - 293s 293ms/step - loss: 114.1252 - accuracy: 0.0342 - val_loss: 113.9611 - val_accuracy: 0.0363
Epoch 4/10
1000/1000 [==============================] - 292s 292ms/step - loss: 113.6619 - accuracy: 0.0382 - val_loss: 113.4772 - val_accuracy: 0.0415
Epoch 5/10
1000/1000 [==============================] - 292s 292ms/step - loss: 113.2129 - accuracy: 0.0426 - val_loss: 113.0457 - val_accuracy: 0.0470
Epoch 6/10
1000/1000 [==============================] - 293s 293ms/step - loss: 112.7843 - accuracy: 0.0461 - val_loss: 112.7196 - val_accuracy: 0.0500
Epoch 7/10
1000/1000 [==============================] - 292s 292ms/step - loss: 112.4479

In [31]:
model1.save('cifar.h5')